In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/머신러닝1/조별활동/와인품질분류/data

/content/drive/MyDrive/Colab Notebooks/머신러닝1/조별활동/와인품질분류/data


In [ ]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [ ]:
test

fixed acidity  volatile acidity  citric acid  residual sugar  \
index                                                                 
0                9.0              0.31         0.48            6.60   
1               13.3              0.43         0.58            1.90   
2                6.5              0.28         0.27            5.20   
3                7.2              0.15         0.39            1.80   
4                6.8              0.26         0.26            2.00   
...              ...               ...          ...             ...   
995              7.1              0.59         0.02            2.30   
996              8.7              0.15         0.30            1.60   
997              8.8              0.66         0.26            1.70   
998              7.0              0.42         0.19            2.30   
999              8.5              0.21         0.26            9.25   

       chlorides  free sulfur dioxide  total sulfur dioxide  density    pH  \
index                                                                        
0          0.043                 11.0                  73.0  0.99380  2.90   
1          0.070                 15.0                  40.0  1.00040  3.06   
2          0.040                 44.0                 179.0  0.99480  3.19   
3          0.043                 21.0                 159.0  0.99480  3.52   
4          0.019                 23.5                  72.0  0.99041  3.16   
...          ...                  ...                   ...      ...   ...   
995        0.082                 24.0                  94.0  0.99744  3.55   
996        0.046                 29.0                 130.0  0.99420  3.22   
997        0.074                  4.0                  23.0  0.99710  3.15   
998        0.071                 18.0                  36.0  0.99476  3.39   
999        0.034                 73.0                 142.0  0.99450  3.05   

       sulphates  alcohol   type  
index                             
0           0.38     11.6  white  
1           0.49      9.0    red  
2           0.69      9.4  white  
3           0.47     10.0  white  
4           0.47     11.8  white  
...          ...      ...    ...  
995         0.53      9.7    red  
996         0.38      9.8  white  
997         0.74      9.2    red  
998         0.56     10.9    red  
999         0.37     11.4  white  

[1000 rows x 12 columns]

# 전처리

## label_encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(train['type'])
train['type'] = label_encoder.transform(train['type'])
test['type'] = label_encoder.transform(test['type'])

## 스케일링

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [ ]:
minmax_col_list = ['fixed acidity', 'citric acid' ,'chlorides','free sulfur dioxide','density','pH','sulphates']
robust_col_list = ['volatile acidity' ,'total sulfur dioxide', 'residual sugar','alcohol']

In [ ]:
min_max_scaler = MinMaxScaler()
scaled_train_data = min_max_scaler.fit_transform(train[minmax_col_list])
scaled_test_data = min_max_scaler.transform(test[minmax_col_list])

train[minmax_col_list] = scaled_train_data
test[minmax_col_list] = scaled_test_data

In [ ]:
robust_scaler = RobustScaler()
scaled_train_data = robust_scaler.fit_transform(train[robust_col_list])
scaled_test_data = robust_scaler.transform(test[robust_col_list])

train[robust_col_list] = scaled_train_data
test[robust_col_list] = scaled_test_data

## upsampling

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
X = train.drop('quality', axis=1)  # 입력 변수
y = train['quality']  # 타겟 변수

smote = SMOTE(k_neighbors=4)
X_resampled, y_resampled = smote.fit_resample(X, y)


train_resampled = pd.concat([X_resampled, y_resampled], axis=1)
train_resampled

ValueError: ignored

# 모델링

## randomforest

In [ ]:
y = train_resampled['quality'] #퀄리티 예측해야할거
X = train_resampled.drop('quality', axis=1) #예측해야할거빼고 나머지

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
model = RandomForestClassifier()

stratified_split = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
# cv_accuracy = []

n_iter = 0
for train_idx, test_idx in stratified_split.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

    #모델 학습
    model.fit(X_train, y_train)

y_pred=model.predict(test)

# 제출파일

In [ ]:
submission = pd.read_csv('sample_submission.csv')

In [ ]:
submission

In [ ]:
submission['quality'] = y_pred

In [ ]:
submission.to_csv('processing_randomforest.csv', index=False)